<a href="https://www.kaggle.com/code/prabhatxoplore/predictstudentscore-sgd-regressor?scriptVersionId=292476832" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Start

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/playground-series-s6e1/train.csv',index_col=0)
test = pd.read_csv('/kaggle/input/playground-series-s6e1/test.csv',index_col=0)

In [3]:
df.isna().sum()

age                 0
gender              0
course              0
study_hours         0
class_attendance    0
internet_access     0
sleep_hours         0
sleep_quality       0
study_method        0
facility_rating     0
exam_difficulty     0
exam_score          0
dtype: int64

In [4]:
df.nunique()

age                   8
gender                3
course                7
study_hours         792
class_attendance    617
internet_access       2
sleep_hours          66
sleep_quality         3
study_method          5
facility_rating       3
exam_difficulty       3
exam_score          805
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 630000 entries, 0 to 629999
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   age               630000 non-null  int64  
 1   gender            630000 non-null  object 
 2   course            630000 non-null  object 
 3   study_hours       630000 non-null  float64
 4   class_attendance  630000 non-null  float64
 5   internet_access   630000 non-null  object 
 6   sleep_hours       630000 non-null  float64
 7   sleep_quality     630000 non-null  object 
 8   study_method      630000 non-null  object 
 9   facility_rating   630000 non-null  object 
 10  exam_difficulty   630000 non-null  object 
 11  exam_score        630000 non-null  float64
dtypes: float64(4), int64(1), object(7)
memory usage: 62.5+ MB


# Train test split data

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,:11],df.iloc[:,11],test_size = 0.2,random_state=2)

In [7]:
X_train.sample()

,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty
id,,,,,,,,,,,
212133,19,female,ba,6.63,85.0,yes,7.5,poor,coaching,low,easy


# Standard Scaling

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

num_cols = scaler.fit_transform((X_train.select_dtypes(include=['number'])))
cat_cols = X_train.select_dtypes(include=['object','category'])

test_num_col = scaler.transform((X_test.select_dtypes(include=['number'])))
test_cat_col = X_test.select_dtypes(include=['object','category'])

test_exn = scaler.transform((test.select_dtypes(include='number')))
test_exc = test.select_dtypes(include=['object','category'])

# One Hot Encoding

In [9]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first',sparse_output=False)

cat_col_ohe = ohe.fit_transform(cat_cols)
test_cat_col_ohe = ohe.transform(test_cat_col)


test_ex_col = ohe.transform(test_exc)


cat_col_ohe_idx = ohe.get_feature_names_out(cat_cols.columns.values)

In [10]:
print(cat_col_ohe_idx)
cat_col_ohe_idx.shape

['gender_male' 'gender_other' 'course_b.sc' 'course_b.tech' 'course_ba'
 'course_bba' 'course_bca' 'course_diploma' 'internet_access_yes'
 'sleep_quality_good' 'sleep_quality_poor' 'study_method_group study'
 'study_method_mixed' 'study_method_online videos'
 'study_method_self-study' 'facility_rating_low' 'facility_rating_medium'
 'exam_difficulty_hard' 'exam_difficulty_moderate']


(19,)

# Merging two categorical and numerical columns

In [11]:
oh_df = np.hstack((num_cols,cat_col_ohe))
toh_df = np.hstack((test_num_col,test_cat_col_ohe))
tex_df = np.hstack((test_exn,test_ex_col))
# pd.DataFrame(hdf).shape

In [12]:
pd.DataFrame(toh_df).shape

(126000, 23)

# SGD Regressor

In [13]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score, root_mean_squared_error

In [14]:
sreg = SGDRegressor(max_iter=100,learning_rate='constant',eta0=0.0001)
sreg.fit(oh_df,y_train)
y_pred = sreg.predict(toh_df)
print(r2_score(y_test,y_pred),'\n')
print(root_mean_squared_error(y_test,y_pred))

0.7771487558143011 

8.904530548336542


In [15]:
y_pred = sreg.predict(tex_df)

# Submission

In [16]:
submission = pd.DataFrame({
    'id':test.index.values,
    'exam_score':y_pred
})

In [17]:
submission.to_csv('submission.csv',index = False)